# Transfer Learning on the Deepweeds Dataset with MobileNetV3

In this notebook, in order to classify different types of weed images based on the **Deepweeds Dataset** which contains RGB colour images of 9 weed species in Australia.
We'll use the MobileNetV3, since it will offer a good foundation.  

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from collections import Counter

## Importing the Dataset

We'll now import the DeepWeeds data, perform train - test split and optimize it. We'll import it using tfds to skip some code.

In [2]:
(data_train, data_val, data_test), data_info = tfds.load("deep_weeds", split=["train[:70%]", "train[70%:85%]", "train[85%:]"], as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/deep_weeds/incomplete.WL43C4_3.0.0/deep_weeds-train.tfrecord*...:   0%|   …

Dataset deep_weeds downloaded and prepared to /root/tensorflow_datasets/deep_weeds/3.0.0. Subsequent calls will reuse this data.


In [3]:
print(data_info)

tfds.core.DatasetInfo(
    name='deep_weeds',
    full_name='deep_weeds/3.0.0',
    description="""
    The DeepWeeds dataset consists of 17,509 images capturing eight different weed species native to Australia in situ with neighbouring flora.The selected weed species are local to pastoral grasslands across the state of Queensland.The images were collected from weed infestations at the following sites across Queensland: "Black River", "Charters Towers",  "Cluden", "Douglas", "Hervey Range", "Kelso", "McKinlay" and "Paluma".
    """,
    homepage='https://github.com/AlexOlsen/DeepWeeds',
    data_dir='/root/tensorflow_datasets/deep_weeds/3.0.0',
    file_format=tfrecord,
    download_size=469.32 MiB,
    dataset_size=469.99 MiB,
    features=FeaturesDict({
        'image': Image(shape=(256, 256, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=9),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False

In [4]:
NUM_CLASSES = data_info.features["label"].num_classes
IMG_SIZE = 224 # Since ResNet50 was trained on 224x224 images
BATCH_SIZE = 32

print(data_info.features["label"].names)

['Chinee apple', 'Lantana', 'Parkinsonia', 'Parthenium', 'Prickly acacia', 'Rubber vine', 'Siam weed', 'Snake weed', 'Negative']


## Preprocessing and Data Augmentation

We'll now write functions to resize the images, normalize, and do some data augmentation for better training.

In [5]:
def preprocess(image, label):
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  image = preprocess_input(image)
  return image, label

In [6]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.3),
])

def augment(image, label):
  return data_augmentation(image, training=True), label

## Build Data Pipeline

We'll now load the data into `tf.data` pipelinfe.

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = data_train.map(preprocess, num_parallel_calls=AUTOTUNE).map(augment, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_data = data_val.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_data = data_test.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

## Class Weights

Since our dataset is heavily biased towards negative samples, we should implement class weights to creectly balance the model.

In [8]:
train_labels = []

for _, label in tfds.as_numpy(data_train):
  train_labels.append(label)

train_labels = np.array(train_labels)

label_counts = Counter(train_labels)

class_weights = {
    cls: len(train_labels) / (NUM_CLASSES * count)
    for cls, count in label_counts.items()
}


## Squeeze and Excitation

We'll use a technique known as squeeze and excitiation to basically assign different weights adaptively to different channels.

In [9]:
def se_block(x, reduction=16):
    filters = x.shape[-1]
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(filters // reduction, activation="relu")(se)
    se = layers.Dense(filters, activation="sigmoid")(se)
    se = layers.Reshape((1, 1, filters))(se)
    return layers.Multiply()([x, se])

## Building the Model

We'll now build our model architecture to fine tune. We'll first go with transfer learning.

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(
    include_top=False,
    weights = "imagenet",
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
)

base_model.trainable = False # Feature Extraction Only

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = se_block(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MobileNetV3Large    │ (None, 7, 7, 960) │  2,996,352 │ input_layer_2[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 960)       │          0 │ MobileNetV3Large… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 60)        │     57,660 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 960)       │     58,560 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 960) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_8          │ (None, 7, 7, 960) │          0 │ MobileNetV3Large… │
│ (Multiply)          │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 960)       │          0 │ multiply_8[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 960)       │      3,840 │ global_average_p… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    246,016 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 9)         │      2,313 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,364,741 (12.84 MB)

 Trainable params: 366,469 (1.40 MB)

 Non-trainable params: 2,998,272 (11.44 MB)

## Training the Model

We'll now train our model.

The reason we went with partial layers and not full model is because resnet50 is too large to train.

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [13]:
history = model.fit(train_data, validation_data=val_data, epochs=15, class_weight=class_weights, callbacks=[tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)])

Epoch 1/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 174s 368ms/step - accuracy: 0.3946 - loss: 1.6523 - val_accuracy: 0.5554 - val_loss: 1.2101
Epoch 2/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 138s 334ms/step - accuracy: 0.5958 - loss: 0.8435 - val_accuracy: 0.6742 - val_loss: 0.8935
Epoch 3/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 140s 328ms/step - accuracy: 0.6616 - loss: 0.7014 - val_accuracy: 0.6391 - val_loss: 1.0327
Epoch 4/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 137s 330ms/step - accuracy: 0.6970 - loss: 0.6012 - val_accuracy: 0.7019 - val_loss: 0.8226
Epoch 5/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 141s 329ms/step - accuracy: 0.7034 - loss: 0.5839 - val_accuracy: 0.7103 - val_loss: 0.8468
Epoch 6/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 137s 329ms/step - accuracy: 0.7221 - loss: 0.5690 - val_accuracy: 0.7103 - val_loss: 0.8625
Epoch 7/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 137s 330ms/step - accuracy: 0.7307 - loss: 0.5102 - val_accuracy: 0.6578 - val_loss: 1.0559
Epoch 8/15
383/383 ━━━━━━━━━━━━━━━━━━━━ 137s 330ms/step - accuracy: 0.7441 -

## Evalutaion Metrics

We'll evaluate the model on the test set and print classification report and confusion matrix.

In [14]:
y_true = []
y_pred = []

for images, labels in test_data:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(tf.argmax(preds, axis=1).numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━

In [15]:
print(classification_report(
    y_true,
    y_pred,
    target_names=data_info.features["label"].names,
    digits=4
))

                precision    recall  f1-score   support

  Chinee apple     0.6429    0.5928    0.6168       167
       Lantana     0.5437    0.8954    0.6765       153
   Parkinsonia     0.8232    0.9255    0.8713       161
    Parthenium     0.8416    0.5822    0.6883       146
Prickly acacia     0.7109    0.8929    0.7916       168
   Rubber vine     0.5785    0.9524    0.7198       147
     Siam weed     0.7236    0.8623    0.7869       167
    Snake weed     0.5226    0.8901    0.6585       182
      Negative     0.9467    0.6921    0.7997      1335

      accuracy                         0.7578      2626
     macro avg     0.7037    0.8095    0.7344      2626
  weighted avg     0.8112    0.7578    0.7635      2626



In [16]:
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[ 99  11   0   0   0   5   2  41   9]
 [  2 137   1   0   0   2   3   2   6]
 [  1   0 149   0  10   0   0   0   1]
 [  2   3   1  85  13   7   2  24   9]
 [  1   1   4   0 150   2   1   6   3]
 [  2   0   0   0   0 140   0   1   4]
 [  0   5   0   0   0   2 144   1  15]
 [  2   7   2   0   1   0   3 162   5]
 [ 45  88  24  16  37  84  44  73 924]]


## Conclusion


The results indicate a solid classification performance with a accuracy of 75.8% and weighted F1-score of 0.76 indicating good precision and recall. Several weed categories have high recall and the negative class has a good precision. Thus the model is good but can be further refined.